In [1]:
%pip install stability-sdk
import getpass, os

# Our host url should not be prepended with "https" nor should it have a trailing slash.
os.environ['STABILITY_HOST'] = 'grpc.stability.ai:443'

# Sign up for an account at the following link to get an API Key.
# https://platform.stability.ai/

# Click on the following link once you have created an account to be taken to your API Key.
# https://platform.stability.ai/account/keys

# Paste your API Key below after running this cell.

os.environ['STABILITY_KEY'] = getpass.getpass('Enter your API Key')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 43.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.62.0
    Uninstalling grpcio-1.62.0:
      Successfully uninstalled grpcio-1.62.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.
Enter your API Key··········


In [15]:
import pandas as pd
import io
import os
import warnings

from IPython.display import display
from PIL import Image
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation
from transformers import CLIPProcessor, CLIPModel
import numpy as np

In [16]:
df1 = pd.read_csv('sample.csv')

print(len(df1['Value'][0].split(', ')))
df = pd.read_csv('sample.csv')
df1.head()

512


,id,prompt,Value
0,19,the scarab from the moon knight marvel series....,"0.32214, 0.25303, 0.98277, 0.26791, 0.65892, 0..."
1,34,"close - up shot portrait, imogen poots as virt...","0.62186, 0.18243, 0.91075, 0.21062, 0.17187, 0..."
2,67,a majestic firedragon creature fighting a figh...,"0.35074, 0.53031, 0.21727, 0.84621, 0.77336, 0..."
3,91,anya taylor - joy emma watson as nuragic warri...,"0.13854, 0.39552, 0.7468, 0.56531, 0.37403, 0...."
4,85,"a corgi casting a magic spell, d & d character...","0.64797, 0.94036, 0.72118, 0.6321, 0.13903, 0...."


In [17]:
# Set up our connection to the API.
stability_api = client.StabilityInference(
    key=os.environ['STABILITY_KEY'], # API Key reference.
    verbose=True, # Print debug messages.
    engine="stable-diffusion-xl-1024-v1-0", # Set the engine to use for generation.
    # Check out the following link for a list of available engines: https://platform.stability.ai/docs/features/api-parameters#engine
)

def callApi(prompt, seed, steps, cfg_scale, width, height, sampler):
    answers = stability_api.generate(
      prompt=prompt,
      seed=seed, # If a seed is provided, the resulting generated image will be deterministic.
                      # What this means is that as long as all generation parameters remain the same, you can always recall the same image simply by generating it again.
                      # Note: This isn't quite the case for CLIP Guided generations, which we tackle in the CLIP Guidance documentation.
      steps=steps, # Amount of inference steps performed on image generation. Defaults to 30.
      cfg_scale=cfg_scale, # Influences how strongly your generation is guided to match your prompt.
                    # Setting this value higher increases the strength in which it tries to match your prompt.
                    # Defaults to 7.0 if not specified.
      width=width, # Generation width, defaults to 512 if not included.
      height=height, # Generation height, defaults to 512 if not included.
      sampler=sampler # Choose which sampler we want to denoise our generation with.
                                                  # Defaults to k_dpmpp_2m if not specified. Clip Guidance only supports ancestral samplers.
                                                  # generation.SAMPLER_K_EULER (Available Samplers: ddim, plms, k_euler, k_euler_ancestral, k_heun, k_dpm_2, k_dpm_2_ancestral, k_dpmpp_2s_ancestral, k_lms, k_dpmpp_2m, k_dpmpp_sde)
    )

    # Set up our warning to print to the console if the adult content classifier is tripped.
    # If adult content classifier is not tripped, display generated image.
    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.finish_reason == generation.FILTER:
                warnings.warn(
                    "Your request activated the API's safety filters and could not be processed."
                    "Please modify the prompt and try again.")
            if artifact.type == generation.ARTIFACT_IMAGE:
                img = Image.open(io.BytesIO(artifact.binary))

    return img

INFO:stability_sdk.client:Opening channel to grpc.stability.ai:443
INFO:stability_sdk.client:Channel opened to grpc.stability.ai:443


In [18]:
def generate_embedding(prompt, seed, steps, cfg_scale, width, height, sampler):
    img = callApi(prompt, seed, steps, cfg_scale, width, height, sampler)
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    inputs = processor(images=img, return_tensors="pt")
    image_embeddings = model.get_image_features(**inputs)
    return image_embeddings

In [19]:
df['embedding'] = np.nan

In [20]:
seed = 123123
steps = 30
cfg_scale = 7
width = 512
height = 512
sampler = generation.SAMPLER_K_DPMPP_2M

for i in range(len(df)):
    prompt = df.iloc[i]['prompt']
    embedding = generate_embedding(prompt, seed, steps, cfg_scale, width, height, sampler)
    embedding = embedding.detach()
    df['embedding'][i] = str(embedding.tolist()[0][:512])[2:-2]

INFO:stability_sdk.client:Sending request.
INFO:stability_sdk.client:Got answer  with artifact types ['ARTIFACT_IMAGE'] in 6.85s
<ipython-input-20-7f3af8bde920>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['embedding'][i] = str(embedding.tolist()[0][:512])[2:-2]
INFO:stability_sdk.client:Sending request.
INFO:stability_sdk.client:Got answer  with artifact types ['ARTIFACT_IMAGE'] in 7.00s
INFO:stability_sdk.client:Sending request.
INFO:stability_sdk.client:Got answer  with artifact types ['ARTIFACT_IMAGE'] in 5.75s
INFO:stability_sdk.client:Sending request.
INFO:stability_sdk.client:Got answer  with artifact types ['ARTIFACT_IMAGE'] in 5.85s
INFO:stability_sdk.client:Sending request.
INFO:stability_sdk.client:Got answer  with artifact types ['ARTIFACT_IMAGE'] in 5.33s
INFO:stability_sdk.client:

In [21]:
df.head()

,id,prompt,Value,embedding
0,19,the scarab from the moon knight marvel series....,"0.32214, 0.25303, 0.98277, 0.26791, 0.65892, 0...","0.3143784999847412, 0.23003990948200226, 0.152..."
1,34,"close - up shot portrait, imogen poots as virt...","0.62186, 0.18243, 0.91075, 0.21062, 0.17187, 0...",".06949915736913681, 0.2086513191461563, 0.0288..."
2,67,a majestic firedragon creature fighting a figh...,"0.35074, 0.53031, 0.21727, 0.84621, 0.77336, 0...",".15591007471084595, 0.10945466160774231, -0.24..."
3,91,anya taylor - joy emma watson as nuragic warri...,"0.13854, 0.39552, 0.7468, 0.56531, 0.37403, 0....","0.18964682519435883, -0.05994897335767746, -0...."
4,85,"a corgi casting a magic spell, d & d character...","0.64797, 0.94036, 0.72118, 0.6321, 0.13903, 0....",".031052540987730026, 0.5678229928016663, -0.03..."


In [24]:
def cos_sim(x, y):
  return np.dot(x, y)/(np.norm(x)*np.norm(y))

sum = 0
for i in range(len(df)):
  sum += cos_sim(df['embedding'][i], df['Value'][i]) > 0.9
  print(cos_sim(df['embedding'][i], df['Value'][i]))
print(sum/len(df))

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U10805'), dtype('<U4563')) -> None

In [25]:
df.drop(['Value'], axis=1, inplace=True)

In [26]:
df.columns = ['id', 'prompt', 'Value']

In [27]:
df.head()

,id,prompt,Value
0,19,the scarab from the moon knight marvel series....,"0.3143784999847412, 0.23003990948200226, 0.152..."
1,34,"close - up shot portrait, imogen poots as virt...",".06949915736913681, 0.2086513191461563, 0.0288..."
2,67,a majestic firedragon creature fighting a figh...,".15591007471084595, 0.10945466160774231, -0.24..."
3,91,anya taylor - joy emma watson as nuragic warri...,"0.18964682519435883, -0.05994897335767746, -0...."
4,85,"a corgi casting a magic spell, d & d character...",".031052540987730026, 0.5678229928016663, -0.03..."


In [28]:
df.to_csv('ThirdPass.csv', index=False)

In [ ]:
df = pd.read_csv('firstPass.csv')


In [ ]:
len(df['Value'][0].split(', '))

512